# Wurun Library - Real Use Cases

Practical examples for async OpenAI API calls in Jupyter notebooks.

In [ ]:
from wurun import Wurun
import asyncio
import time

endpoint = ""
model_name = "Phi-4-mini-reasoning"
deployment_name = "Phi-4-mini-reasoning"

api_key = ""

# Setup - run once per kernel session
await Wurun.setup(
    endpoint=endpoint,  # or your Azure endpoint
    api_key=api_key,
    deployment_name=deployment_name # or your model name
)

## Use Case 1: Single Question

In [ ]:
# Simple single question
messages = [{"role": "user", "content": "Explain Python asyncio in 2 sentences"}]
answer = await Wurun.ask(messages)
print(answer)

## Use Case 2: Batch Code Review

In [3]:
# Review multiple code snippets concurrently
code_snippets = [
    "def factorial(n): return 1 if n <= 1 else n * factorial(n-1)",
    "for i in range(len(items)): print(items[i])",
    "result = [x for x in data if x > 0]"
]

review_messages = [
    [{"role": "system", "content": "Review this code for best practices. Be concise."},
     {"role": "user", "content": f"Code: {code}"}]
    for code in code_snippets
]

start = time.time()
reviews = await Wurun.run_gather(review_messages, concurrency=3)
print(f"Reviewed {len(code_snippets)} snippets in {time.time() - start:.2f}s\n")

for code, review in zip(code_snippets, reviews):
    print(f"Code: {code}")
    print(f"Review: {review}\n")

Reviewed 3 snippets in 21.72s

Code: def factorial(n): return 1 if n <= 1 else n * factorial(n-1)
Review: [ERROR] InternalServerError: Error code: 500 - {'error': {'code': 'Internal Server Error', 'message': 'An error occurred during the request processing: All connection attempts failed | Type: ConnectError | Traceback (most recent call last):||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpx/_transports/default.py", line 66, in map_httpcore_exceptions||    yield||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpx/_transports/default.py", line 366, in handle_async_request||    resp = await self._pool.handle_async_request(req)||           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request||    raise exc from None||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpcore/_async/conn

## Use Case 3: Data Analysis Questions

In [4]:
# Analyze dataset with multiple questions
dataset_info = "Sales data: 10k records, columns: date, product, price, quantity, region"

analysis_questions = [
    "What are 3 key metrics to calculate?",
    "Suggest 2 visualization types", 
    "What data quality checks should I run?",
    "Recommend a machine learning approach"
]

analysis_messages = [
    [{"role": "system", "content": f"You're analyzing: {dataset_info}. Be specific and actionable."},
     {"role": "user", "content": q}]
    for q in analysis_questions
]

await Wurun.print_qna_ordered(analysis_messages, concurrency=4)

Q: What are 3 key metrics to calculate?
A: [ERROR] InternalServerError: Error code: 500 - {'error': {'code': 'Internal Server Error', 'message': 'An error occurred during the request processing: All connection attempts failed | Type: ConnectError | Traceback (most recent call last):||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpx/_transports/default.py", line 66, in map_httpcore_exceptions||    yield||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpx/_transports/default.py", line 366, in handle_async_request||    resp = await self._pool.handle_async_request(req)||           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request||    raise exc from None||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request||    respons

## Use Case 4: Content Generation Pipeline

In [5]:
# Generate content for multiple topics
topics = ["Machine Learning", "Cloud Computing", "Data Science"]

content_tasks = []
for topic in topics:
    content_tasks.extend([
        [{"role": "user", "content": f"Write a 50-word intro about {topic}"}],
        [{"role": "user", "content": f"List 3 key benefits of {topic}"}],
        [{"role": "user", "content": f"Suggest 2 learning resources for {topic}"}]
    ])

print(f"Generating content for {len(topics)} topics ({len(content_tasks)} total tasks)...")
await Wurun.print_as_ready(content_tasks, concurrency=5)

Generating content for 3 topics (9 total tasks)...


KeyError: <coroutine object as_completed.<locals>._wait_for_one at 0x750306e6b100>

## Use Case 5: Error Handling & Retries

In [6]:
# Robust call with metadata and custom retry settings
messages = [{"role": "user", "content": "Explain quantum computing"}]

result, meta = await Wurun.ask(
    messages,
    attempts=3,
    initial_backoff=1.0,
    max_backoff=10.0,
    return_meta=True
)

print(f"Response: {result}")
print(f"Latency: {meta['latency']:.2f}s, Retries: {meta['retries']}")

Response: [ERROR] InternalServerError: Error code: 500 - {'error': {'code': 'Internal Server Error', 'message': 'An error occurred during the request processing: All connection attempts failed | Type: ConnectError | Traceback (most recent call last):||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpx/_transports/default.py", line 66, in map_httpcore_exceptions||    yield||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpx/_transports/default.py", line 366, in handle_async_request||    resp = await self._pool.handle_async_request(req)||           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request||    raise exc from None||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request||    response = await connection.handle_async

## Use Case 6: Conversation Chain

In [7]:
# Build conversation context
conversation = [
    {"role": "system", "content": "You're a Python expert. Keep responses under 100 words."},
    {"role": "user", "content": "What's the difference between list and tuple?"}
]

# First response
response1 = await Wurun.ask(conversation)
print(f"Q: {conversation[-1]['content']}")
print(f"A: {response1}\n")

# Add to conversation and ask follow-up
conversation.extend([
    {"role": "assistant", "content": response1},
    {"role": "user", "content": "Give me a code example"}
])

response2 = await Wurun.ask(conversation)
print(f"Q: {conversation[-1]['content']}")
print(f"A: {response2}")

Q: What's the difference between list and tuple?
A: [ERROR] InternalServerError: Error code: 500 - {'error': {'code': 'Internal Server Error', 'message': 'An error occurred during the request processing: All connection attempts failed | Type: ConnectError | Traceback (most recent call last):||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpx/_transports/default.py", line 66, in map_httpcore_exceptions||    yield||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpx/_transports/default.py", line 366, in handle_async_request||    resp = await self._pool.handle_async_request(req)||           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request||    raise exc from None||  File "/opt/microsoft-maas/billingproxy/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request||  

## Cleanup

In [8]:
# Clean up resources when done
await Wurun.close()
print("Wurun client closed")

Wurun client closed
